In [31]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, datediff, max as spark_max, min as spark_min, count, sum as spark_sum, lit
from pyspark.sql.types import DoubleType, DateType

In [3]:
spark = SparkSession.builder \
    .appName("hym") \
    .master("local[2]") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

## Carga y limpieza de datos

### Cargar la data en Spark

In [11]:
transactions_df = spark.read.parquet("/home/jovyan/data/transactions.parquet", header=True, inferSchema=True)

In [12]:
customers_df = spark.read.csv("/home/jovyan/data/customers.csv", header=True, inferSchema=True)

### Explorar y limpiar la data

In [13]:
# Ver estructura de los datos

customers_df.printSchema()
transactions_df.printSchema()

customers_df.show(5)
transactions_df.show(5)

root
 |-- customer_id: string (nullable = true)
 |-- FN: double (nullable = true)
 |-- Active: double (nullable = true)
 |-- club_member_status: string (nullable = true)
 |-- fashion_news_frequency: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- postal_code: string (nullable = true)

root
 |-- t_dat: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- article_id: long (nullable = true)
 |-- price: double (nullable = true)
 |-- sales_channel_id: long (nullable = true)

+--------------------+----+------+------------------+----------------------+---+--------------------+
|         customer_id|  FN|Active|club_member_status|fashion_news_frequency|age|         postal_code|
+--------------------+----+------+------------------+----------------------+---+--------------------+
|00000dbacae5abe5e...|NULL|  NULL|            ACTIVE|                  NONE| 49|52043ee2162cf5aa7...|
|0000423b00ade9141...|NULL|  NULL|            ACTIVE|                  NONE| 25

In [14]:
# Eliminar duplicados

customers_df = customers_df.dropDuplicates(["customer_id"])
transactions_df = transactions_df.dropDuplicates()

In [19]:
# Manejo de valores nulos

# Clientes sin ID no sirven
customers_df = customers_df.na.drop(subset=["customer_id"])

# Transacciones sin customer_id o date no son válidas
transactions_df = transactions_df.na.drop(subset=["customer_id", "t_dat", "price"])

In [20]:
# Asegurar tipos correctos

transactions_df = transactions_df.withColumn("price", col("price").cast("double")) \
                                 .withColumn("t_dat", col("t_dat").cast("date"))

In [21]:
# Filtrar precios inválidos

transactions_df = transactions_df.filter(col("price") > 0)

## Unir clientes y transacciones

In [23]:
df = transactions_df.join(customers_df, on="customer_id", how="inner")

## Análisis LRFM

### Definir la fecha de referencia

Para calcular Recency (R) necesitamos una fecha de referencia. Lo más común es usar la última fecha de compra en el dataset

In [28]:
# Obtener la última fecha en el dataset
max_date = transactions_df.agg(spark_max("t_dat")).collect()[0][0]
print("Fecha máxima en transacciones:", max_date)

Fecha máxima en transacciones: 2020-09-22


### Calcular las métricas LRFM

Agrupamos por customer_id y calculamos cada métrica:

In [29]:
lrfm_df = transactions_df.groupBy("customer_id").agg(
    # L = tiempo de vida (última compra - primera compra)
    (datediff(spark_max("t_dat"), spark_min("t_dat"))).alias("L"),
    
    # R = días desde última compra hasta la fecha de referencia
    datediff(lit(max_date), spark_max("t_dat")).alias("R"),
    
    # F = número de compras realizadas
    count("*").alias("F"),
    
    # M = gasto total del cliente
    spark_sum("price").alias("M")
)

lrfm_df.show(10)


+--------------------+---+---+---+-------------------+
|         customer_id|  L|  R|  F|                  M|
+--------------------+---+---+---+-------------------+
|fde6f7236ab9c9618...|162|571| 12|  0.203203389830508|
|20d8178fd4d25d012...|706| 26| 41|  0.823220338983049|
|56211faa7c8b6f289...|728|  4| 54| 0.9037118644067762|
|0037dbd391c8b1535...|300|370|  7|0.24818644067796575|
|052107ad48bdd5ee8...|  0|696|  7|0.14938983050847432|
|0720207dca00bce45...|707| 11| 98| 3.4057457627118586|
|07ba2a6d232c1b069...|  0|710|  2| 0.0528644067796609|
|09eee30bd6fc147a5...|548|149| 14|0.31637288135593167|
|0c16b1c8dbda2d0f4...|662|  3| 75| 2.1115254237288106|
|0c4bba00ec7edf56e...|272|432| 16| 0.3005932203389825|
+--------------------+---+---+---+-------------------+
only showing top 10 rows



Interpretación:

Primer clientes, cliente activo por un tiempo limitado (L medio), pero lleva casi lleva tiempo sin comprar (R alto). Frecuencia y gasto moderados → cliente “dormido”.

Segundo cliente,cliente con larga vida (L alto), compró hace poco (R bajo), bastante frecuente y buen gasto → cliente leal y activo.

Tercer cliente, cliente muy longevo, compra muy reciente, altísima frecuencia y gasto → cliente VIP.

Cuarto cliente, historial intermedio, pero lleva más de un año sin comprar (R alto), baja frecuencia y gasto → cliente en riesgo de pérdida.

### Segmentación de clientes

In [33]:
segmented_df = lrfm_df.withColumn(
    "Segmento",
    when((col("R") <= 30) & (col("F") >= 10) & (col("M") > 1000), "Cliente VIP")
    .when((col("R") <= 30) & (col("F") < 10), "Cliente Nuevo")
    .when((col("R") > 60), "Cliente en Riesgo")
    .otherwise("Cliente Regular")
)

segmented_df.show(10)

+--------------------+---+---+---+-------------------+-----------------+
|         customer_id|  L|  R|  F|                  M|         Segmento|
+--------------------+---+---+---+-------------------+-----------------+
|fde6f7236ab9c9618...|162|571| 12|  0.203203389830508|Cliente en Riesgo|
|20d8178fd4d25d012...|706| 26| 41|  0.823220338983049|  Cliente Regular|
|56211faa7c8b6f289...|728|  4| 54| 0.9037118644067762|  Cliente Regular|
|0037dbd391c8b1535...|300|370|  7|0.24818644067796575|Cliente en Riesgo|
|052107ad48bdd5ee8...|  0|696|  7|0.14938983050847432|Cliente en Riesgo|
|0720207dca00bce45...|707| 11| 98| 3.4057457627118586|  Cliente Regular|
|07ba2a6d232c1b069...|  0|710|  2| 0.0528644067796609|Cliente en Riesgo|
|09eee30bd6fc147a5...|548|149| 14|0.31637288135593167|Cliente en Riesgo|
|0c16b1c8dbda2d0f4...|662|  3| 75| 2.1115254237288106|  Cliente Regular|
|0c4bba00ec7edf56e...|272|432| 16| 0.3005932203389825|Cliente en Riesgo|
+--------------------+---+---+---+-----------------